# 2nd Level Model Structure: Reboiler

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from pyomo import environ as pe
from global_sets.component import m
from utility.display_utility import trans_product_mole, trans_product_mass
from utility.model_utility import add_dual, update_dual, check_DOF

# stage construction rules
from physics.energy_reboiler import energy_block_rule
from physics.VLE_reboiler_MPCC_P import VLE_block_rule
from physics.MPCC_P import P_NCP_block_rule, P_Reg_block_rule, P_pf_block_rule

model = pe.ConcreteModel()

# Global Sets (Inlet / Outlet)

In [3]:
model.inlet = pe.Set(initialize=['in'])
model.outlet = pe.Set(initialize=['out','P'])
model.stream = model.inlet | model.outlet

# Global Variables

In [4]:
# Tray Inlet/Outlet Variable
model.x_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.x = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.z = pe.Var(m.COMP_FEED,within=pe.NonNegativeReals)

model.L = pe.Var(model.stream,within=pe.NonNegativeReals)
model.V = pe.Var(model.stream,within=pe.NonNegativeReals)
model.F = pe.Var(within=pe.NonNegativeReals)

model.H_L_ = pe.Var(model.inlet,within=pe.Reals)
model.H_V_ = pe.Var(model.inlet,within=pe.Reals)
model.H_L = pe.Var(within=pe.Reals)
model.H_V = pe.Var(within=pe.Reals)
model.H_F = pe.Var(within=pe.Reals)

# State Variable
model.T = pe.Var(within=pe.NonNegativeReals,bounds=(200+273.15,350+273.15)) # K
model.T_F = pe.Var(within=pe.NonNegativeReals) # K
model.P = pe.Var(within=pe.NonNegativeReals,bounds=(10,30)) # Bar

model.f_V = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)
model.f_L = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)

model.Q_main = pe.Var(within=pe.Reals) # MW

# Construct Individual Blocks

In [5]:
model.energy_block = pe.Block(rule=energy_block_rule)

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| energy_block.dH_F
| energy_block.dH_V
| energy_block.dH_L
| energy_block.dH_vap
--------------------------------------------------



In [6]:
model.VLE_block = pe.Block(rule=VLE_block_rule)

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| VLE_block.P_VLE
| VLE_block.n_ave
| VLE_block.n_ave_cal
| VLE_block.Hen
| VLE_block.Hen0
| VLE_block.gamma
| VLE_block.P_sat
| VLE_block.P_sat_Y
| VLE_block.P_sat_dY_inf
| VLE_block.P_sat_dY0
| VLE_block.Hen_ref
| VLE_block.Hen0_ref
| VLE_block.gamma_ref
| VLE_block.V_L
| VLE_block.V_L_dY_inf
| VLE_block.V_L_dY0
| VLE_block.poynting
--------------------------------------------------



# Standard MESH Equations

## Mass Balance

\begin{equation}
Fz_{i}+\sum_{s\in{inlet}}L_{s}x'_{i,s}+\sum_{s\in{inlet}}V_{s}y'_{i,s}-\sum_{s\in{outlet}}L_{s}x_{i,s}+\sum_{s\in{ouelet}}V_{s}y_{i,s}+R_{i} = 0 \\
i = 1,...NC
\end{equation}

In [7]:
def mass_balance_main_rule(model,i):
    if i in m.COMP_FEED:
        return model.F*model.z[i] + sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
    else:
        return sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
model.mass_balance_main_con = pe.Constraint(m.COMP_TOTAL,rule=mass_balance_main_rule)

## Equilibrium

\begin{align}
&f_{i,V} = f_{i,L} \\
&i = 1,...NC
\end{align}

In [8]:
def VL_equil_rule(model,i):
    return model.f_V[i] == model.f_L[i]
model.VL_equil_con = pe.Constraint(m.COMP_TOTAL,rule=VL_equil_rule)

# MPCC - P-Pspec

In [9]:
model.MPCC = pe.Block(rule = P_pf_block_rule)

> Importing MPCC_P_Reg Blocks......
> Adding the following local variable:
--------------------------------------------------
| MPCC.s_L
| MPCC.s_V
| MPCC.pf
| MPCC.rho
--------------------------------------------------
> Spliting pressure used in VLE



## Summation

\begin{equation}
\sum_ix_{i} = \sum_iy_{i} \\
F + L_{in} + V_{in} + R = L_{out} + V_{out}
\end{equation}

In [10]:
def summation_x_y_rule(model):
    return sum(model.x[i] for i in m.COMP_TOTAL) == sum(model.y[i] for i in m.COMP_TOTAL)
model.summation_x_y_con = pe.Constraint(rule=summation_x_y_rule)

def summation_total_mass_rule(model):
    return model.F + sum(model.L[s] + model.V[s] for s in model.inlet)\
            - sum(model.L[s] + model.V[s] for s in model.outlet) == 0
model.summation_total_mass_con = pe.Constraint(rule=summation_total_mass_rule)

## Energy Balance

\begin{equation}
F H_f+\sum_{s\in{inlet}}L_{s}H_{l,s}+\sum_{s\in{inlet}}V_{s}H_{v,s}-\sum_{s\in{outlet}}L_{s}H_{l,s}-\sum_{s\in{outlet}}V_{s}H_{v,s}+Q = 0
\end{equation}

In [11]:
def heat_balance_main_rule(model):
    return model.F*model.H_F + sum(model.L[s]*model.H_L_[s] + model.V[s]*model.H_V_[s] for s in model.inlet) \
            + model.Q_main - sum(model.L[s]*model.H_L + model.V[s]*model.H_V for s in model.outlet) == 0
model.heat_balance_main_con = pe.Constraint(rule=heat_balance_main_rule)

# Testing

In [12]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


# Load a flash liquid composition as feed

In [13]:
with open('../saved_solutions/reactive_flash_MPCC_P_pf_200C.pickle', 'rb') as f:
    C200 = pickle.load(f)

## Fixing In/Product Redundent Flow

In [14]:
model.y_.fix(0)
for i in m.COMP_TOTAL:
    model.x_['in',i].fix(C200.Solution.Variable['x[{}]'.format(i)]['Value'])
# model.L['in'].fix(1)
model.L['P'].fix(0)
model.V['in'].fix(0)
model.V['P'].fix(0)
model.H_V_.fix(0)
model.H_L_.fix(C200.Solution.Variable['H_L']['Value'])

## Fixing Model Parameters

In [15]:
model.P.fix(20)
model.T_F.fix(200+273.15)
model.F.fix(0)
model.z.fix(0)
model.VLE_block.n_ave.fix(20)
model.Q_main.fix(0)

In [16]:
check_DOF(pe,model)

Active Equality Constraints:	 957
Active Inequality Constraints:	 1
Active Variables:		 1132
Fixed Variables:		 171
DOF:				 4


## Fix T or to fix Q? we have found that Maximize against an upper bound is most reliable

In [17]:
model.T.fix(350+273.15)

In [18]:
# model.obj = pe.Objective(expr = model.T ,sense=pe.maximize)
# model.obj = pe.Objective(expr = model.MPCC.pf,sense=pe.minimize)
model.obj = pe.Objective(expr = - model.L['in'] - model.MPCC.pf ,sense=pe.maximize)

In [19]:
opt = pe.SolverFactory('ipopt')

opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma86'

opt.options['linear_system_scaling '] = 'mc19'
opt.options['linear_scaling_on_demand '] = 'no'
# opt.options['recalc_y'] = 'yes'
# opt.options['bound_relax_factor'] = 0
# opt.options['halt_on_ampl_error'] = 'yes'

opt.options['max_iter'] = 7000
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling =mc19
linear_scaling_on_demand =no
max_iter=7000


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with

  82  1.9202962e-01 1.17e-02 1.65e+06  -1.0 2.27e+01    -  9.65e-01 9.62e-01f  1
  83  2.0017813e-01 5.08e-04 1.54e+09  -1.0 6.34e-01    -  9.90e-01 1.00e+00h  1
  84  2.0009984e-01 4.50e-02 8.36e+10  -1.0 1.98e+01    -  9.93e-01 4.39e-01f  2
  85  1.9999959e-01 4.11e-02 1.53e+11  -1.0 3.35e+00    -  1.00e+00 1.00e+00h  1
  86  1.9999959e-01 1.76e-03 2.09e+15  -1.0 1.57e+01    -  1.00e+00 1.00e+00h  1
  87  1.9999959e-01 1.21e-01 7.37e+15  -1.0 3.08e+02    -  2.81e-01 1.60e-01f  1
  88  1.9999959e-01 1.37e+01 7.13e+15  -1.0 7.13e-05  20.0 1.39e-04 1.00e+00f  1
  89  1.9950454e-01 2.85e+00 3.21e+21  -1.0 6.98e-02  19.5 1.00e+00 7.81e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  90  1.9996522e-01 2.40e-04 4.04e+20  -1.0 1.52e-02  19.0 1.98e-02 9.91e-01h  1
  91  1.9999959e-01 6.43e-10 1.12e+18  -1.7 1.39e-04  18.6 1.00e+00 1.00e+00h  1
  92  1.9999959e-01 6.43e-10 1.12e+18  -5.7 1.55e+12    -  1.00e+00 2.02e-09f  6
  93  1.8783509e-01 6.04e-10

ValueError: Cannot load a SolverResults object with bad status: error

In [ ]:
model.Q_main.value

In [ ]:
model.T.value

In [ ]:
# model.solutions.store_to(results)
# with open('../saved_solutions/reboiler_MPCC_P_pf_200C.pickle','wb') as f:
#     pickle.dump(results,f)

In [ ]:
print('Component\t\tLiquid: {:.4f}\t\t\tVapor: {:.4f}\t\t\tlog K'.format(model.L['out'].value,model.V['out'].value))
print('-'*108)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:8.4%}\t\t\t{:8.4%}\t\t\t{:.4f}'.format(model.x[i].value,model.y[i].value,np.log10(model.y[i].value/model.x[i].value)))

# Iterative Solve for Data Analysis

In [ ]:
update_dual(pe,model)

In [ ]:
opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6

In [ ]:
rf_data = {}
rf_data['T'] = []; rf_data['Q'] = []; rf_data['V'] = []; rf_data['L'] = []; 
rf_data['y_CO'] = []; rf_data['y_H2'] = []; rf_data['y_CO2'] = []; rf_data['y_H2O'] = [];
rf_data['f_V_CO'] = []; rf_data['f_V_H2'] = [];
    
Trange = np.arange(350+273.15,199+273.15,-5)

for Tub in Trange:
    model.T.setub(Tub)
    results = opt.solve(model,tee=False)
    update_dual(pe,model)
    # print('-'*72)
    print('Solved, Solution T = {:.2f} K\t|\tV = {:.5f}\t|\tL = {:.5f}\t|'.format(model.T.value,model.V['out'].value,model.L['out'].value))
    # print('-'*72)
    rf_data['T'].append(model.T.value)
    rf_data['Q'].append(model.Q_main.value)
    rf_data['V'].append(model.V['out'].value)
    rf_data['L'].append(model.L['out'].value)    
    
    
    rf_data['y_H2O'].append(model.y['H2O'].value)
    rf_data['y_CO'].append(model.y['CO'].value)
    rf_data['y_H2'].append(model.y['H2'].value)
    rf_data['y_CO2'].append(model.y['CO2'].value)
    
    rf_data['f_V_CO'].append(model.f_V['CO'].value)
    rf_data['f_V_H2'].append(model.f_V['H2'].value)

# VLE Validation using AspenPlus

### Feed (Reactor Effluent)

In [ ]:
print('Temperature (C): \t{:.2f}\t\t'.format(model.T.value-273.15))
print('Total FLow (kmol/s): \t{:.2f}\t\t'.format(model.L['out'].value+model.V['out'].value))
print('-'*72)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:.4%}'.format((model.x[i].value*model.L['out'].value\
        +model.y[i].value*model.V['out'].value)/(model.L['out'].value+model.V['out'].value)))

### Phase Separation

In [ ]:
print('Component\t\tLiquid: {:.4f}\t\t\tVapor: {:.4f}\t\t\tlog K'.format(model.L['out'].value,model.V['out'].value))
print('-'*108)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:8.4%}\t\t\t{:8.4%}\t\t\t{:.4f}'.format(model.x[i].value,model.y[i].value,np.log10(model.y[i].value/model.x[i].value)))

In [ ]:
list(zip(rf_data['T'],rf_data['Q']))